In [21]:
import os
import zipfile
import pandas as pd
import numpy as np
from PIL import Image
from glob import glob
from tqdm import tqdm
from pprint import pprint
import matplotlib.pyplot as plt
import time

from collections import Sequence
# %matplotlib inline
import cv2

from PIL import Image
#import pickle5 as pickle

import os.path as osp
import matplotlib.cm as cm

# ML libs
import torchvision
from torchvision import transforms
from torchvision import models
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.optim import lr_scheduler
from torch import optim
from torchvision.utils import make_grid
from torch.utils.data import Dataset
from torch.nn import functional as F

In [22]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

batch_size    learning_rate    num_epochs    train_acc    test_acc  
64            0.0001           20            98.1200      74.5400 (adam)
64            0.00001          20            96.3800      67.2400 (adam)
64            0.0001           25            98.5680      74.5800 (adam)
64            0.01             25            87.0360      74.1700 (SGD+scheduler)
32            0.01             40            99.9680      82.0300 (SGD+scheduler)
64            0.001            25            88.9740      74.3800 (SGD+momentum=0.9+scheduler)


In [23]:
batch_size = 32
#learning_rate = 0.0001
num_epochs = 40

In [24]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    # transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    ])

In [25]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                            shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root="./data",train=False,transform=transform, download=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                            shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [26]:
shape = trainloader.dataset.data.shape
print(shape)

(50000, 32, 32, 3)


In [27]:
device = torch.device('mps')
myNet = torchvision.models.vgg11()
myNet.classifier[6] = nn.Linear(in_features=4096, out_features=10, bias=True)
myNet.to(device)

# myNet = torch.load("/Users/songjunzhe/Desktop/python/ml_attack/myvgg11_cifar10_97_7240.pt")
# myNet.to(device)

criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(myNet.parameters(), lr = learning_rate)
optimizer = optim.SGD(myNet.parameters(), lr=0.01, weight_decay=5e-3, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=5, gamma=0.4, last_epoch=-1)



In [28]:
print("myNet training\n")
n_total_steps = len(trainloader)

for epoch in tqdm(range(num_epochs)):
    correct = 0
    for i, (images, labels) in enumerate(trainloader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = myNet(images)
        loss = criterion(outputs, labels)
        correct += (outputs.argmax(1) == labels).type(torch.float).sum().item()

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # correct += (outputs == labels).float().sum()

        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
    correct /= len(trainloader.dataset)        
    print(f"Acc: {(100 * correct):>0.4f}\n")
    scheduler.step()
print('Finished Training')

myNet training



  0%|          | 0/40 [00:00<?, ?it/s]

Epoch [1/40], Step [100/1563], Loss: 2.1799
Epoch [1/40], Step [200/1563], Loss: 1.9928
Epoch [1/40], Step [300/1563], Loss: 2.1461
Epoch [1/40], Step [400/1563], Loss: 1.7700
Epoch [1/40], Step [500/1563], Loss: 1.7094
Epoch [1/40], Step [600/1563], Loss: 1.4423
Epoch [1/40], Step [700/1563], Loss: 1.6360
Epoch [1/40], Step [800/1563], Loss: 1.5609
Epoch [1/40], Step [900/1563], Loss: 1.4832
Epoch [1/40], Step [1000/1563], Loss: 1.6533
Epoch [1/40], Step [1100/1563], Loss: 1.5339
Epoch [1/40], Step [1200/1563], Loss: 1.6360
Epoch [1/40], Step [1300/1563], Loss: 1.2485
Epoch [1/40], Step [1400/1563], Loss: 1.2846
Epoch [1/40], Step [1500/1563], Loss: 1.2626


  2%|▎         | 1/40 [06:44<4:22:37, 404.03s/it]

Acc: 36.5880

Epoch [2/40], Step [100/1563], Loss: 1.2242
Epoch [2/40], Step [200/1563], Loss: 1.0205
Epoch [2/40], Step [300/1563], Loss: 1.2813
Epoch [2/40], Step [400/1563], Loss: 1.2353
Epoch [2/40], Step [500/1563], Loss: 1.2175
Epoch [2/40], Step [600/1563], Loss: 0.9993
Epoch [2/40], Step [700/1563], Loss: 1.1917
Epoch [2/40], Step [800/1563], Loss: 1.2654
Epoch [2/40], Step [900/1563], Loss: 1.0278
Epoch [2/40], Step [1000/1563], Loss: 0.9112
Epoch [2/40], Step [1100/1563], Loss: 1.6848
Epoch [2/40], Step [1200/1563], Loss: 1.3818
Epoch [2/40], Step [1300/1563], Loss: 0.8171
Epoch [2/40], Step [1400/1563], Loss: 1.0372
Epoch [2/40], Step [1500/1563], Loss: 1.4853


...

Acc: 99.9680

Epoch [39/40], Step [100/1563], Loss: 0.0128
Epoch [39/40], Step [200/1563], Loss: 0.0146
Epoch [39/40], Step [300/1563], Loss: 0.0184
Epoch [39/40], Step [400/1563], Loss: 0.0143
Epoch [39/40], Step [500/1563], Loss: 0.0129
Epoch [39/40], Step [600/1563], Loss: 0.0082
Epoch [39/40], Step [700/1563], Loss: 0.0203
Epoch [39/40], Step [800/1563], Loss: 0.0588
Epoch [39/40], Step [900/1563], Loss: 0.0122
Epoch [39/40], Step [1000/1563], Loss: 0.0084
Epoch [39/40], Step [1100/1563], Loss: 0.0096
Epoch [39/40], Step [1200/1563], Loss: 0.0138
Epoch [39/40], Step [1300/1563], Loss: 0.0140
Epoch [39/40], Step [1400/1563], Loss: 0.0221
Epoch [39/40], Step [1500/1563], Loss: 0.0265


 98%|█████████▊| 39/40 [4:05:17<06:20, 380.03s/it]

Acc: 99.9560

Epoch [40/40], Step [100/1563], Loss: 0.0095
Epoch [40/40], Step [200/1563], Loss: 0.0073
Epoch [40/40], Step [300/1563], Loss: 0.0176
Epoch [40/40], Step [400/1563], Loss: 0.0283
Epoch [40/40], Step [500/1563], Loss: 0.0156
Epoch [40/40], Step [600/1563], Loss: 0.0201
Epoch [40/40], Step [700/1563], Loss: 0.0190
Epoch [40/40], Step [800/1563], Loss: 0.0132
Epoch [40/40], Step [900/1563], Loss: 0.0096
Epoch [40/40], Step [1000/1563], Loss: 0.0090
Epoch [40/40], Step [1100/1563], Loss: 0.0090
Epoch [40/40], Step [1200/1563], Loss: 0.0145
Epoch [40/40], Step [1300/1563], Loss: 0.0205
Epoch [40/40], Step [1400/1563], Loss: 0.0178
Epoch [40/40], Step [1500/1563], Loss: 0.0160


100%|██████████| 40/40 [4:11:31<00:00, 377.29s/it]

Acc: 99.9680

Finished Training


In [29]:
print("myNet testing\n")

myNet.eval()

size = len(testloader.dataset)
correct = 0
with torch.no_grad():
    for i, (images, labels) in enumerate(testloader):
        images = images.to(device)
        labels = labels.to(device)
        pred = myNet(images)
        correct += (pred.argmax(1)==labels).type(torch.float).sum().item()
    
correct /= size
print(f"Test Accuracy: {(100 * correct):>0.4f}\n")

myNet testing

Test Accuracy: 82.0300



In [30]:
myNet.to('cpu')
torch.save(myNet,"./myvgg11_cifar10.pt")